<a href="https://colab.research.google.com/github/alexus987/R/blob/master/bq_dsb_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Data Insights to DSB BigQuery copy</h1>


[link text](https://) yoyo

# Authenticate

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


# BQ Operations

Copy all tables of a dataset

In [15]:
tables = !bq --project='telia-ddi-dk-dev' ls 'telia-ddi-dk-dev:alexey' | awk '{print $1}'
tables.pop(0)
tables.pop(0)

for table in tables:
  print('Copying table: '+table)
  !bq --project='telia-ddi-dk-dev' cp -f 'telia-ddi-dk-dev:alexey.$table' 'apollo-dsb:odm.$table'

print(' ')
!bq --project='telia-ddi-dk-dev' --format=pretty ls 'apollo-dsb:odm'

Copying table: 1hourly_odm_ltm_mixed
Waiting on bqjob_r1a1c2e254bf07257_0000016e4b98b08f_1 ... (0s) Current status: DONE   
Table 'telia-ddi-dk-dev:alexey.1hourly_odm_ltm_mixed' successfully copied to 'apollo-dsb:odm.1hourly_odm_ltm_mixed'
Copying table: 1hourly_odm_ltm_oal2_dal2
Waiting on bqjob_r3cfc44626b294f50_0000016e4b98c55f_1 ... (0s) Current status: DONE   
Table 'telia-ddi-dk-dev:alexey.1hourly_odm_ltm_oal2_dal2' successfully copied to 'apollo-dsb:odm.1hourly_odm_ltm_oal2_dal2'
Copying table: 1hourly_via_via_ltm2_20190801_agg3_via4
Waiting on bqjob_r6693e3fab731aa69_0000016e4b98d659_1 ... (0s) Current status: DONE   
Table 'telia-ddi-dk-dev:alexey.1hourly_via_via_ltm2_20190801_agg3_via4' successfully copied to 'apollo-dsb:odm.1hourly_via_via_ltm2_20190801_agg3_via4'
Copying table: 1hourly_via_via_mixed_20190801_agg3_via4
Waiting on bqjob_r64a368e83f06017a_0000016e4b98e72b_1 ... (0s) Current status: DONE   
Table 'telia-ddi-dk-dev:alexey.1hourly_via_via_mixed_20190801_agg3_via4

Copy one table

In [0]:
!bq --project=telia-ddi-dk-dev cp -f 'telia-ddi-dk-dev:DSB_export.1hourly_odm_ltm_oal4_dal4' 'apollo-dsb:odm.1hourly_odm_ltm_oal4_dal4'

Delete tables from Dataset

In [14]:
tables = !bq --project='telia-ddi-dk-dev' ls 'apollo-dsb:odm' | awk '{print $1}'
tables.pop(0)
tables.pop(0)

for table in tables:
  if table != 'dk_ltm_mixed':
    print('Deleting table: '+table)
    !bq --project='telia-ddi-dk-dev' rm -f -t 'apollo-dsb:odm.$table'

print(' ')
!bq --project='telia-ddi-dk-dev' --format=pretty ls 'apollo-dsb:odm'

Deleting table: 05hourly2_odm_ltm_mixed_20190801
Deleting table: 05hourly2_odm_ltm_mixed_20190801_start_time_only
Deleting table: 05hourly2_odm_ltm_oal4_dal4_20190801
Deleting table: 05hourly_wodm_line_mixed_20190801
Deleting table: 05hourly_wodm_line_mixed_20190801_agg2_via3
Deleting table: 05hourly_wodm_line_mixed_20190801_agg2_via4
Deleting table: 12hourly2_odm_ltm_mixed_20190801
Deleting table: 12hourly2_odm_ltm_oal4_dal4_20190801
Deleting table: 12hourly_wodm_line_mixed_20190801
Deleting table: 12hourly_wodm_line_mixed_20190801_agg2_via3
Deleting table: 12hourly_wodm_line_mixed_20190801_agg2_via4
Deleting table: 1hourly_odm_ltm_mixed
Deleting table: 1hourly_odm_ltm_mixed_20190801_start_time_only
Deleting table: 1hourly_odm_ltm_oal4_dal4
Deleting table: 1hourly_wodm_line_mixed_20190801
Deleting table: 1hourly_wodm_line_mixed_20190801_agg2_v4
Deleting table: 1hourly_wodm_line_mixed_20190801_agg2_via3
Deleting table: 1hourly_wodm_line_mixed_20190801_agg2_via4
Deleting table: 1hourly_

# Re-Generate maps


In [8]:
!bq load --autodetect --replace DSB_import.via_points_kit_stretches_v3 gs://telia-ddi-dk-dev-dsb-reference/via_points_kit_stretches_v3.34.csv

Waiting on bqjob_r5acc5636da6e822e_0000016e4b2e8bc9_1 ... (3s) Current status: DONE   


In [9]:
%%bigquery --project=telia-ddi-dk-dev  
CREATE OR REPLACE TABLE  `telia-ddi-dk-dev.DSB_export.via_points_kit_stretches_v3` AS
	SELECT
	v.StretchIdentifier,
	v.FirstStation,
	v.LastStation,
	ST_GEOGPOINT(v.lon_1 , v.lat_1) as StartPoint,
	ST_GEOGPOINT(v.lon_2 , v.lat_2) as FinishPoint,
	ST_MAKELINE(ST_GEOGPOINT(v.lon_1 , v.lat_1), ST_GEOGPOINT(v.lon_2 , v.lat_2)) as ViaLine,
	ST_DISTANCE(ST_GEOGPOINT(v.lon_1 , v.lat_1), ST_GEOGPOINT(v.lon_2 , v.lat_2)) as ViaLine_Distance  
FROM `DSB_import.via_points_kit_stretches_v3` v

""


# Re-Generate ODMs

In [13]:
# 1hourly ODM Via

%%bigquery --project=telia-ddi-dk-dev  
create or replace table `alexey.1hourly_wodm_line_mixed_20190801_agg3_via3` as
--create or replace table `apollo-dsb.odm.1hourly_wodm_line_mixed_20190801_agg3_via3` as
--create or replace table `DSB_export.1hourly_wodm_line_mixed_20190801_agg3_via3` as
--create or replace table `mobility_insights.1hourly_wodm_line_mixed_20190801_agg3_via3` as


WITH transits AS (
  SELECT
    t.transit_id,
    t.identifier,
    go.code AS origin_area_code,
    gd.code AS destination_area_code,
    vp.StretchIdentifier ,
    TIMESTAMP_TRUNC(t.start_time, HOUR) AS transit_start_hour,
    DATETIME_TRUNC(t.local_start_time, HOUR) AS transit_local_start_hour,
    TIMESTAMP_TRUNC(t.end_time, HOUR) AS transit_end_hour,
    DATETIME_TRUNC(t.local_end_time, HOUR) AS transit_local_end_hour
  FROM `telia-ddi-dk-prod.mobility_signals.transits_20190801` t
  INNER JOIN `telia-ddi-dk-prod.mobility_signals.quality_tiers_20190801` tier ON t.identifier = tier.identifier
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  go ON st_covers(go.geo, t.origin_point)
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  gd ON st_covers(gd.geo, t.destination_point )
  INNER JOIN `DSB_export.via_points_kit_stretches_v3` vp 
    ON ST_INTERSECTS(t.line, vp.ViaLine )
  WHERE tier.rating != "C"
  # Filter out transits with low density
  #AND (IFNULL(num_clusters_in_transit, 0) + 2) / (duration /60 /60) >= 6 
)
, ex_customers AS (
  SELECT
    (1 / p_observation) * m_cluster_adjuster * m_device_reducer AS m_customers
  FROM `telia-ddi-dk-prod.extrapolation.signal_probabilities_day_20190801`
)
, ex_identifier AS (
  SELECT
    identifier,
    observed_market_share
  FROM `telia-ddi-dk-prod.mobility_signals.areas_of_habit_20190801` h
  INNER JOIN `telia-ddi-dk-prod.maps.administrative_level_4` ma ON ST_CONTAINS(ma.geo, h.point)
  INNER JOIN `telia-ddi-dk-prod.extrapolation.administrative_level_3_baseline_market_share` ms ON ms.admin_level_3_code = ma.admin_level_3_code
  WHERE
    h.aoh_classification = 'HOME'
)
# New p_quality_transit
, ex_transit AS (
  SELECT
    EXTRACT(hour FROM start_time) AS start_hour,
    1 / (COUNT(DISTINCT CASE WHEN (IFNULL(num_clusters_in_transit, 0) + 2) / (duration /60 /60) >= 6 THEN identifier ELSE NULL END) / COUNT(DISTINCT identifier)) AS m_quality_transits
  FROM `telia-ddi-dk-prod.mobility_signals.transits_20190902`
  INNER JOIN `telia-ddi-dk-prod.mobility_signals.quality_tiers_20190902` USING(identifier) 
  WHERE rating != 'C'
  GROUP BY
    start_hour
  ORDER BY 
    start_hour
)
, ex_transits_num AS (
  SELECT
    origin_area_code,
    --derived_waypoint_grid_code,
    StretchIdentifier ,
    destination_area_code,
    transit_start_hour,
    transit_local_start_hour,
    transit_end_hour ,
    transit_local_end_hour,
    COUNT(*) AS devices,
    CAST(ROUND(COUNT(*) * ANY_VALUE(m_customers)) AS INT64) AS customers,
    # inflated "raw" number,
    CAST(ROUND(COUNT(*) * ANY_VALUE(m_customers) * ANY_VALUE(m_quality_transits)) AS INT64) AS new_customers,
    CAST(ROUND(SUM((1/observed_market_share) * m_customers)) AS INT64) AS people,
    # inflated people number
    CAST(ROUND(SUM((1/observed_market_share) * m_customers * m_quality_transits)) AS INT64) AS new_people
  FROM transits t, ex_customers 
  LEFT JOIN ex_identifier i
    ON i.identifier = t.identifier
  LEFT JOIN ex_transit
    ON extract(hour from t.transit_start_hour) = ex_transit.start_hour 
  GROUP BY
    origin_area_code,
    --derived_waypoint_grid_code,
    destination_area_code,
    StretchIdentifier,
    transit_start_hour,
    transit_local_start_hour,
    transit_end_hour,
    transit_local_end_hour
  HAVING
    SUM((1/observed_market_share) * m_customers) >= 0
)
SELECT
   n.origin_area_code,
  -- n.derived_waypoint_grid_code,
   n.StretchIdentifier,
   n.destination_area_code,
   n.transit_start_hour,
   n.transit_local_start_hour,
   n.transit_end_hour,
   n.transit_local_end_hour,
   --CAST(ROUND(n.devices, 0) AS INT64)       AS devices,
   --CAST(ROUND(n.customers, 0) AS INT64)     AS customers,
   --CAST(ROUND(n.new_customers, 0) AS INT64) AS new_customers,
   --CAST(ROUND(n.people, 0) AS INT64)        AS people,
   CAST(ROUND(n.new_people, 0) AS INT64)    AS people
FROM ex_transits_num n
WHERE n.new_people > 5
    
     


""


In [12]:
# 24H Via  

%%bigquery --project=telia-ddi-dk-dev 

--create or replace table `mobility_insights.24hourly_wodm_line_mixed_20190801_agg2_via3` as --non_extr
create or replace table `alexey.24hourly_wodm_line_mixed_20190801_agg3_via3` as
--create or replace table `apollo-dsb.odm.24hourly_wodm_line_mixed_20190801_agg2_via3` as

WITH transits AS (
  SELECT
    t.transit_id,
    t.identifier,
    go.code AS origin_area_code,
    gd.code AS destination_area_code,
    vp.StretchIdentifier ,
    TIMESTAMP_TRUNC(start_time, DAY) AS utc_day
  FROM `telia-ddi-dk-prod.mobility_signals.transits_20190801` t
  INNER JOIN `telia-ddi-dk-prod.mobility_signals.quality_tiers_20190801` tier ON t.identifier = tier.identifier
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  go ON st_covers(go.geo, t.origin_point)
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  gd ON st_covers(gd.geo, t.destination_point )
  INNER JOIN `DSB_export.via_points_kit_stretches_v3` vp 
    ON ST_INTERSECTS(t.line, vp.ViaLine )  WHERE tier.rating != "C"
)
, ex_customers AS (
  SELECT
    (1 / p_observation) * m_cluster_adjuster * m_device_reducer AS m_customers
  FROM `telia-ddi-dk-prod.extrapolation.signal_probabilities_day_20190801`
)

# New p_quality_transit
, ex_transit_hourly AS (
  SELECT
    EXTRACT(hour FROM start_time) AS start_hour,
    1 / (COUNT(DISTINCT CASE WHEN (IFNULL(num_clusters_in_transit, 0) + 2) / (duration /60 /60) >= 6 THEN identifier ELSE NULL END) / COUNT(DISTINCT identifier)) AS m_quality_transits
  FROM `telia-ddi-dk-prod.mobility_signals.transits_20190902`
  INNER JOIN `telia-ddi-dk-prod.mobility_signals.quality_tiers_20190902` USING(identifier) 
  WHERE rating != 'C'
  GROUP BY
    start_hour
  ORDER BY 
    start_hour
)

, ex_transit as (
SELECT 
AVG(m_quality_transits) as m_quality_transits
from ex_transit_hourly
)

, ex_identifier AS (
  SELECT
    identifier,
    observed_market_share
  FROM `telia-ddi-dk-prod.mobility_signals.areas_of_habit_20190801` h
  INNER JOIN `telia-ddi-dk-prod.maps.administrative_level_4` ma ON ST_CONTAINS(ma.geo, h.point)
  INNER JOIN `telia-ddi-dk-prod.extrapolation.administrative_level_3_baseline_market_share` ms ON ms.admin_level_3_code = ma.admin_level_3_code
  WHERE
    h.aoh_classification = 'HOME'
)
, ex_transits_num AS (
  SELECT
    origin_area_code,
    utc_day,
    StretchIdentifier ,
    destination_area_code,
    COUNT(*) AS devices,
    CAST(ROUND(COUNT(*) * ANY_VALUE(m_customers)) AS INT64) AS customers,
    # inflated "raw" number,
    CAST(ROUND(COUNT(*) * ANY_VALUE(m_customers) * ANY_VALUE(m_quality_transits)) AS INT64) AS new_customers,
    CAST(ROUND(SUM((1/observed_market_share) * m_customers)) AS INT64) AS people,
    # inflated people number
    CAST(ROUND(SUM((1/observed_market_share) * m_customers * m_quality_transits)) AS INT64) AS new_people
    FROM transits t, ex_customers e, ex_transit
  LEFT JOIN ex_identifier i
    ON i.identifier = t.identifier
  GROUP BY
    origin_area_code,
    destination_area_code,
    utc_day,
    StretchIdentifier
  HAVING
    SUM((1/observed_market_share) * m_customers) >= 0

)

SELECT
   n.origin_area_code,
   n.StretchIdentifier,
   n.destination_area_code,
   N.utc_day as date,
   '' as transit_start_hour_utc,
   '' as transit_end_hour_utc,
--   CAST(ROUND(n.customers , 0) AS INT64)     AS customers,
--   CAST(ROUND(n.devices  , 0) AS INT64)     AS devices,
--   CAST(ROUND(n.people, 0) AS INT64)     AS people,
  CAST(ROUND(n.new_people, 0) AS INT64)    AS people

FROM ex_transits_num n
WHERE --n.people > 5
 n.new_people > 5

""


# Via Via

In [0]:
# 1H Via Via 

%%bigquery --project=telia-ddi-dk-dev  

create or replace table DSB_export.1hourly_wodm_line_mixed_20190801_agg2_via4 as

WITH transits AS (
  SELECT
    t.transit_id,
    t.identifier,
    go.code AS origin_area_code,
    gd.code AS destination_area_code,
    vp.StretchIdentifier ,
    TIMESTAMP_TRUNC(t.start_time, HOUR) AS transit_start_hour,
    DATETIME_TRUNC(t.local_start_time, HOUR) AS transit_local_start_hour,
    TIMESTAMP_TRUNC(t.end_time, HOUR) AS transit_end_hour,
    DATETIME_TRUNC(t.local_end_time, HOUR) AS transit_local_end_hour
  FROM `telia-ddi-dk-prod.mobility_signals.transits_20190801` t
  INNER JOIN `telia-ddi-dk-prod.mobility_signals.quality_tiers_20190801` tier ON t.identifier = tier.identifier
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  go ON st_covers(go.geo, t.origin_point)
  INNER JOIN `telia-ddi-dk-dev.david.dk_ltm_mixed`  gd ON st_covers(gd.geo, t.destination_point )
  INNER JOIN `DSB_export.via_via_points_kit_stretches_A1_v4`  vp 
    ON ST_INTERSECTS(t.line, vp.ViaLine_o)
    AND ST_INTERSECTS(t.line, vp.ViaLine_d)
  WHERE tier.rating != "C"
)
, ex_customers AS (
  SELECT
    (1 / p_observation) * m_cluster_adjuster * m_device_reducer AS m_customers
  FROM `telia-ddi-dk-prod.extrapolation.signal_probabilities_day_20190801`
)
, ex_identifier AS (
  SELECT
    identifier,
    observed_market_share
  FROM `telia-ddi-dk-prod.mobility_signals.areas_of_habit_20190801` h
  INNER JOIN `telia-ddi-dk-prod.maps.administrative_level_4` ma ON ST_CONTAINS(ma.geo, h.point)
  INNER JOIN `telia-ddi-dk-prod.extrapolation.administrative_level_3_baseline_market_share` ms ON ms.admin_level_3_code = ma.admin_level_3_code
  WHERE
    h.aoh_classification = 'HOME'
)
, ex_transits_num AS (
  SELECT
    origin_area_code,
    --derived_waypoint_grid_code,
    StretchIdentifier ,
    destination_area_code,
    transit_start_hour,
    transit_local_start_hour,
    transit_end_hour ,
    transit_local_end_hour,
    COUNT(*) AS devices,
    CAST(ROUND(COUNT(*) * ANY_VALUE(m_customers)) AS INT64) AS customers,
    CAST(ROUND(SUM((1/observed_market_share) * m_customers)) AS INT64) AS people
  FROM transits t, ex_customers e
  LEFT JOIN ex_identifier i
    ON i.identifier = t.identifier
  GROUP BY
    origin_area_code,
    --derived_waypoint_grid_code,
    destination_area_code,
    StretchIdentifier,
    transit_start_hour,
    transit_local_start_hour,
    transit_end_hour,
    transit_local_end_hour
  HAVING
    SUM((1/observed_market_share) * m_customers) >= 0
)

SELECT
   n.origin_area_code,
  -- n.derived_waypoint_grid_code,
   n.StretchIdentifier,
   n.destination_area_code,
   n.transit_start_hour,
   n.transit_local_start_hour,
   n.transit_end_hour,
   n.transit_local_end_hour,
   --CAST(ROUND(n.devices, 0) AS INT64)    AS devices,
   --CAST(ROUND(n.customers, 0) AS INT64)  AS customers,
   CAST(ROUND(n.people, 0) AS INT64)     AS people
FROM ex_transits_num n
WHERE n.people > 5